In [3]:
# 学习自己做模型，和看模型
# https://www.tensorflow.org/guide/keras/save_and_serialize
import numpy as np
import tensorflow as tf
from tensorflow import keras

class CustomModel(keras.Model):
    def __init__(self, hidden_units):
        super(CustomModel, self).__init__()
        self.hidden_units = hidden_units
        self.dense_layers = [keras.layers.Dense(u) for u in hidden_units]

    def call(self, inputs):
        x = inputs
        for layer in self.dense_layers:
            x = layer(x)
        return x

    def get_config(self):
        return {"hidden_units": self.hidden_units}

    @classmethod
    def from_config(cls, config):
        return cls(**config)


model = CustomModel([16, 16, 10])
# Build the model by calling it
input_arr = tf.random.uniform((1, 5))
outputs = model(input_arr)
model.save("my_model2")

# Option 1: Load with the custom_object argument.
loaded_1 = keras.models.load_model(
    "my_model2", custom_objects={"CustomModel": CustomModel}
)

# Option 2: Load without the CustomModel class.

# Delete the custom-defined model class to ensure that the loader does not have
# access to it.
del CustomModel

loaded_2 = keras.models.load_model("my_model2")
np.testing.assert_allclose(loaded_1(input_arr), outputs)
np.testing.assert_allclose(loaded_2(input_arr), outputs)

print("Original model:", model)
print("Model Loaded with custom objects:", loaded_1)
print("Model loaded without the custom object class:", loaded_2)

INFO:tensorflow:Assets written to: my_model2/assets
Original model: <__main__.CustomModel object at 0x7f8eb4722f70>
Model Loaded with custom objects: <__main__.CustomModel object at 0x7f8eb46eb400>
Model loaded without the custom object class: <keras.saving.saved_model.load.CustomModel object at 0x7f8ec953d610>
